In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

### Reading the Dataset

In [3]:
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("./datasets/uci-diabetes.csv"))

In [4]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

### Selecting Feature from Dataset and casting it into required Dtype

In [5]:
columns  = df.columns

dataset = df.select(col(columns[1]).cast('float'),
                    col(columns[2]).cast('float'),
                    col(columns[3]).cast('string'),
                    col(columns[4]).cast('float'),
                   )

### Feature Transformation

In [6]:
indexer = StringIndexer(inputCol="code", outputCol="codeIndex")
encoder = OneHotEncoder(inputCol="codeIndex", outputCol="codeVec")
vectorAssembler = VectorAssembler(inputCols=['value','codeVec'],
                                  outputCol="features")
#normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
scaler = StandardScaler(inputCol="features", outputCol="features_norm")
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, scaler])
final_df = pipeline.fit(dataset).transform(dataset)

### Splitting the Dataset into traininng data and test data

In [7]:
(training_data, test_data) = final_df.randomSplit([0.8,0.2])

### Building model

In [8]:
lr = LogisticRegression()
param_grid = (ParamGridBuilder()
              .baseOn({lr.labelCol : 'Class'})
              .baseOn([lr.predictionCol, 'pred_lr'])
              .baseOn([lr.featuresCol, 'features_norm'])
              .addGrid(lr.regParam, [0.01,0.1,1,10,20])
              .addGrid(lr.elasticNetParam, [0,1]).build())

evaluator = BinaryClassificationEvaluator(labelCol= 'Class',rawPredictionCol= 'pred_lr')


cv = CrossValidator(estimator=lr, estimatorParamMaps=param_grid, evaluator=evaluator,seed=0,parallelism=4)

### Training the Model

In [9]:
cvModel = cv.fit(training_data)

In [11]:
cvModel.avgMetrics

[0.9996934438777962,
 0.9993273658135964,
 0.9996959523574815,
 0.9540537260109301,
 0.999555214363244,
 0.5,
 0.5556430103040786,
 0.5,
 0.5,
 0.5]

### Evaluating Trained Model Using Different Evaluation Metrics

In [12]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: 'areaUnderROC'})

0.9998123123123124

In [13]:
evaluator.evaluate(cvModel.transform(test_data), {evaluator.metricName : 'areaUnderPR'})

0.9996802046690119

### Best Params

In [14]:
cvModel.bestModel.getRegParam()

0.1

In [15]:
cvModel.bestModel.getElasticNetParam()

0.0

### Saving the Trained Model

In [16]:
path= r'./SavedModel/lr_model'
cvModel.save(path)

### Loading the Trained Model

In [17]:
#lr_model= CrossValidatorModel.read().load(path)

### Result Summary

In [1]:
import pandas as pd

summary_= {
    'areaUnderPR' : 0.9996802046690119,
    'areaUnderROC' :0.9998123123123124
}
summary= pd.Series(summary_)
summary

areaUnderPR     0.999680
areaUnderROC    0.999812
dtype: float64